In [5]:
import math
from pprint import pprint
from math import log, factorial
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, sets, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m

print()

In [78]:
def weighted_paths_in_tree(edges: List[List[int]], queries: List[int]) -> int:
    n = max(max(x, y) for x, y, _ in edges)
    parents = list(range(n+1))
    sizes = [1] * (n + 1)

    def parent(n: int) -> int:
        while n != parents[n]:
            n, parents[n] = parents[n], parents[parents[n]]
        return n

    def union(n: int, m: int):
        pn, pm = parent(n), parent(m)
        if pn == pm:
            return 0
        paths = sizes[pn] * sizes[pm]
        sizes[pn] += sizes[pm]
        parents[pm] = pn
        return paths

    edges.sort(key=itemgetter(2))
    results = [0] * len(queries)
    paths = 0
    start = 0
    for i, q in sorted(enumerate(queries), key=itemgetter(1)):
        while start < len(edges) and edges[start][2] <= q:
            x, y, w = edges[start]
            paths += union(x, y)
            start += 1
        results[i] = paths
    return results

In [79]:
weighted_paths_in_tree([(3, 1, 3), (4, 2, 2), (3, 4, 1)], [3, 3, 2, 0, 2, 0, 3, 0, 3])

[6, 6, 3, 0, 3, 0, 6, 0, 6]

In [76]:
weighted_paths_in_tree([[1, 2, 1], [2, 3, 4]], [3])

[1]

In [77]:
weighted_paths_in_tree(
    [[1, 2, 3], [2, 3, 1], [2, 4, 9], [3, 6, 7], [3, 5, 8], [5, 7, 4]], [1, 3, 5]
)

[1, 3, 4]